ロストランゲージからスクレイピングしたやつ

In [5]:
import requests
from bs4 import BeautifulSoup
import re
def remove_emojis_regex(text):
    emoji_pattern = re.compile("["
               u"\U0001F1E0-\U0001F1FF"  # 国旗
               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
def clean_airline_name(name):
    # 絵文字除去、空白削除、小文字化
    name = remove_emojis_regex(name)
    return name.replace(" ", "").strip().lower()
lost_luggage_url = 'https://luggagelosers.com/'
response = requests.get(lost_luggage_url)
soup = BeautifulSoup(response.text, "html.parser")
table_tags = soup.find('table')
tr_tags = table_tags.find_all('tr')
airlines_lostbags_data = []
for tr in tr_tags:
    td_tags = tr.find_all('td')
    airline_lost_bags = td_tags[-1].get_text(strip=True)
    for td in td_tags:
        airline_tags = td.find_all('a')
        for airline_tag in airline_tags:
            # 前処理を適用
            airlines = clean_airline_name(airline_tag.get_text(strip=True))
            airlines_lostbags_data.append({
                "Airlines": airlines,
                "Lost_luggage": airline_lost_bags
            })
print("luggagelosersから取得したデータ:")
print(airlines_lostbags_data)

luggagelosersから取得したデータ:
[{'Airlines': 'frontierairlines', 'Lost_luggage': '56,273'}, {'Airlines': 'airindia', 'Lost_luggage': '93,793'}, {'Airlines': 'indigo', 'Lost_luggage': '74,752'}, {'Airlines': 'britishairways', 'Lost_luggage': '88,101'}, {'Airlines': 'iberia', 'Lost_luggage': '26,618'}, {'Airlines': 'virginatlantic', 'Lost_luggage': '13,968'}, {'Airlines': 'pakistaninternationalairlines', 'Lost_luggage': '7,267'}, {'Airlines': 'emirates', 'Lost_luggage': '49,120'}, {'Airlines': 'ethiopianairlines', 'Lost_luggage': '13,582'}, {'Airlines': 'aerlingus', 'Lost_luggage': '10,658'}, {'Airlines': 'swiss', 'Lost_luggage': '10,201'}, {'Airlines': 'qatarairways', 'Lost_luggage': '34,961'}, {'Airlines': 'royalairmaroc', 'Lost_luggage': '8,710'}, {'Airlines': 'kenyaairways', 'Lost_luggage': '6,293'}, {'Airlines': 'avianca', 'Lost_luggage': '25,198'}, {'Airlines': 'aircanada', 'Lost_luggage': '23,077'}, {'Airlines': 'tapportugal', 'Lost_luggage': '7,728'}, {'Airlines': 'americanairlines', 'L

OAGのサイトからスクレイピングしたやつ


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
def clean_airline_name_oag(name):
    return clean_airline_name(name)
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get("https://www.oag.com/on-time-performance-global?submissionGuid=979215ee-32ea-46d7-8568-dbb97bd87839")
iframe = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe[src*='https://flo.uri.sh/visualisation/19357084/embed?auto=1']"))
)
driver.switch_to.frame(iframe)
print("ターゲットのiframeに切り替えました。")
oag_data = []
div_tags = driver.find_elements(By.CSS_SELECTOR, "div.tr.body-row")
for div in div_tags:
    ddiv_tags = div.find_elements(By.CSS_SELECTOR, "div.td")
    oag_airlines = ddiv_tags[0].text.strip()
    oag_airlines = clean_airline_name_oag(oag_airlines)  # 前処理適用
    otp = ddiv_tags[-3].text.strip()
    canceled_flights = ddiv_tags[-2].text.strip()
    total_flights = ddiv_tags[-1].text.strip()
    oag_data.append({
        "Airlines": oag_airlines,
        "OTP": otp,
        "Canceled_flights": canceled_flights,
        "Total_flights": total_flights
    })
driver.switch_to.default_content()
driver.quit()
print("OAGから取得したデータ:")
print(oag_data)


/Users/fuchikota/.pyenv/versions/3.12.7/lib/python3.12/json/decoder.py:353: RuntimeWarning: coroutine 'app_async' was never awaited
  obj, end = self.scan_once(s, idx)


ターゲットのiframeに切り替えました。
OAGから取得したデータ:
[{'Airlines': 'hainanairlines', 'OTP': '92.03%', 'Canceled_flights': '0.33%', 'Total_flights': '22637'}, {'Airlines': 'omanair', 'OTP': '91.61%', 'Canceled_flights': '0%', 'Total_flights': '3280'}, {'Airlines': 'royaljordanian', 'OTP': '91.38%', 'Canceled_flights': '0.08%', 'Total_flights': '2664'}, {'Airlines': 'safair', 'OTP': '90.69%', 'Canceled_flights': '0.14%', 'Total_flights': '5651'}, {'Airlines': 'xiamenairlinescompany', 'OTP': '90.65%', 'Canceled_flights': '0.04%', 'Total_flights': '19611'}, {'Airlines': 'copaairlines', 'OTP': '90.08%', 'Canceled_flights': '0.11%', 'Total_flights': '10780'}, {'Airlines': 'saudiarabianairlines', 'OTP': '89.78%', 'Canceled_flights': '0.07%', 'Total_flights': '16964'}, {'Airlines': 'starflyer', 'OTP': '89.75%', 'Canceled_flights': '0%', 'Total_flights': '1947'}, {'Airlines': 'fujidreamairlines', 'OTP': '89.09%', 'Canceled_flights': '0.31%', 'Total_flights': '2230'}, {'Airlines': 'japantransoceanair', 'OTP': '8

In [9]:
import sqlite3
class DBHandler:
    def __init__(self, db_path="flight_data.db"):
        self.db_path = db_path
        self.create_tables()
    def connect(self):
        return sqlite3.connect(self.db_path)
    def create_tables(self):
        conn = self.connect()
        cursor = conn.cursor()
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS luggagelosers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            Airlines TEXT NOT NULL UNIQUE,
            Lost_luggage INTEGER NOT NULL
        )
        """)
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS oag (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            Airlines TEXT NOT NULL UNIQUE,
            OTP TEXT,
            Canceled_flights TEXT,
            Total_flights TEXT
        )
        """)
        conn.commit()
        conn.close()
    def insert_luggagelosers(self, data_list):
        conn = self.connect()
        cursor = conn.cursor()
        for data in data_list:
            try:
                lost = int(data["Lost_luggage"].replace(',', ''))
            except ValueError:
                lost = 0
            cursor.execute("""
            INSERT OR IGNORE INTO luggagelosers (Airlines, Lost_luggage)
            VALUES (?, ?)
            """, (data["Airlines"], lost))
        conn.commit()
        conn.close()
    def insert_oag(self, data_list):
        conn = self.connect()
        cursor = conn.cursor()
        for data in data_list:
            cursor.execute("""
            INSERT OR IGNORE INTO oag (Airlines, OTP, Canceled_flights, Total_flights)
            VALUES (?, ?, ?, ?)
            """, (data["Airlines"], data["OTP"], data["Canceled_flights"], data["Total_flights"]))
        conn.commit()
        conn.close()
    def normalize_airlines(self, table_name):
        conn = self.connect()
        cursor = conn.cursor()
        cursor.execute(f"SELECT id, Airlines FROM {table_name}")
        rows = cursor.fetchall()
        for row in rows:
            record_id, airline_name = row
            normalized_name = clean_airline_name(airline_name)
            cursor.execute(f"""
            UPDATE {table_name}
            SET Airlines = ?
            WHERE id = ?
            """, (normalized_name, record_id))
        conn.commit()
        conn.close()
    def remove_duplicates(self, table_name):
        conn = self.connect()
        cursor = conn.cursor()
        cursor.execute(f"""
        DELETE FROM {table_name}
        WHERE id NOT IN (
            SELECT MIN(id)
            FROM {table_name}
            GROUP BY Airlines
        )
        """)
        conn.commit()
        conn.close()
    def query_joined_data(self):
        """
        両テーブルをJOINして共通の航空会社のデータを取得
        """
        conn = self.connect()
        cursor = conn.cursor()
        query = """
        SELECT DISTINCT l.Airlines,
                        l.Lost_luggage,
                        o.OTP,
                        o.Canceled_flights,
                        o.Total_flights
        FROM luggagelosers l
        JOIN oag o ON l.Airlines = o.Airlines
        """
        cursor.execute(query)
        results = cursor.fetchall()
        conn.close()
        return results

In [10]:
# DBHandlerクラスのインスタンス化
db = DBHandler("flight_data.db")
# データを挿入
db.insert_luggagelosers(airlines_lostbags_data)
db.insert_oag(oag_data)
# データを正規化
db.normalize_airlines("luggagelosers")
db.normalize_airlines("oag")
# 重複データを削除
db.remove_duplicates("luggagelosers")
db.remove_duplicates("oag")
# JOIN結果の確認
joined_data = db.query_joined_data()
print("JOINしたデータ（正規化・重複除外後）:")
for row in joined_data:
    print(row)
print("一致している航空会社数:", len(joined_data))


JOINしたデータ（正規化・重複除外後）:
('aerlingus', 10658, '77.74%', '2.26%', '8106')
('aerolineasargentinas', 1446, '84.04%', '0.25%', '9627')
('aeromexico', 8214, '84.54%', '0.57%', '16247')
('airarabia', 971, '73.74%', '0.12%', '6635')
('aircanada', 23077, '61.71%', '2.68%', '30936')
('aireuropa', 3367, '82.54%', '0.16%', '5639')
('airfrance', 15442, '68.49%', '1.42%', '23306')
('airindia', 93793, '70.98%', '0.02%', '23300')
('alaskaairlines', 13529, '77.98%', '1.25%', '36768')
('americanairlines', 114298, '76.9%', '1.02%', '182456')
('britishairways', 88101, '69.03%', '3.21%', '24873')
('brusselsairlines', 1930, '76.44%', '0.69%', '5052')
('deltaairlines', 70762, '79.97%', '0.35%', '144281')
('easyjet', 14994, '70.8%', '0.6%', '43862')
('egyptair', 4811, '45.01%', '0.47%', '6874')
('emirates', 49120, '79.56%', '0.03%', '14812')
('etihadairways', 8661, '81.58%', '0.28%', '7896')
('finnair', 2410, '61.22%', '2.5%', '9156')
('flydubai', 481, '76.53%', '0.32%', '10456')
('gulfair', 963, '83.21%', '0%'